<a href="https://colab.research.google.com/github/esds-fiocruz/incidencia_tuberculose_SINAM/blob/main/TUB_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download das biblioteca do PySUS
!pip install pysus
!pip install --upgrade openpyxl

In [1]:
#Importação das bibliotecas
import os
import pandas as pd
import numpy as np
from pysus.ftp.databases.sinan import SINAN
from openpyxl import load_workbook, Workbook

In [2]:
#Carregando as informações do SINAN
sinan = SINAN().load()
files = sinan.get_files(dis_code="TUBE", year=[2023,2022, 2021, 2020, 2019])
tube_2023, tube_2022, tube_2021 , tube_2020, tube_2019  = files

In [3]:
sinan.describe(tube_2023)

{'name': 'TUBEBR19.dbc',
 'disease': 'Tuberculose',
 'year': 2019,
 'size': '5.2 MB',
 'last_update': '2024-04-22 04:23PM'}

In [4]:
# Gerando a estruta de arquivos e obtendo os dados do SINAM
lista_tub = [tube_2023,tube_2022, tube_2021, tube_2020, tube_2019]
for tub in lista_tub:
   nome = str(tub)
   ano = nome[6:8]
   if not os.path.isfile(f'/content/TUB_CN/20{ano}/TUB_CN_20{ano}.csv'):
      os.makedirs(f"./TUB_CN/20{ano}")
      parquet = tub.download()
      df = parquet.to_dataframe()
      df.to_csv(f'/content/TUB_CN/20{ano}/TUB_CN_20{ano}.csv', index=False)

In [5]:
#Filtragem dos anos
def gera_anos(anos):
    if anos is None:
        anos = [2023,2022,2021,2020,2019]
    elif type(anos) is not list:
        anos = [anos]
    else:
        anos = anos
    return anos

In [6]:
#Conversão dos códigos UF's
def gera_ufs(ufs):
  dict_uf = {
        "AC": 12,
        "AL": 27,
        "AP": 16,
        "AM": 13,
        "BA": 29,
        "CE": 23,
        "DF": 53,
        "ES": 32,
        "GO": 52,
        "MA": 21,
        "MT": 51,
        "MS": 50,
        "MG": 31,
        "PA": 15,
        "PB": 25,
        "PR": 41,
        "PE": 26,
        "PI": 22,
        "RJ": 33,
        "RN": 24,
        "RS": 43,
        "RO": 11,
        "RR": 14,
        "SC": 42,
        "SP": 35,
        "SE": 28,
        "TO": 17
    }

  if ufs is None:
      return list(dict_uf.values())
  elif type(ufs) is not list:
      lista_ufs =[dict_uf[ufs]]
      return lista_ufs
  else:
    for uf in ufs:
      lista_ufs =[]
      for key in ufs:
         lista_ufs.append(dict_uf[key])
      return lista_ufs


In [7]:
#Leitura do dados da polulação oriundos do ibge
def obtem_df_ibge(arquivo):
    df_ibge = pd.read_excel(arquivo)
    df_ibge = df_ibge[["CODIGO_UF", "POP_ANT"]]
    df_ibge = df_ibge.dropna()
    df_ibge = df_ibge.sort_values(by=["CODIGO_UF"])
    return df_ibge

In [8]:
#Função principal para a criação da planilha com o coeficiente de incidência de casos novos de tuberculose
def cria_df(anos=None,ufs=None):
    lista_anos = gera_anos(anos)
    lista_ufs = gera_ufs(ufs)

    wb = Workbook()
    wb.save("/content/TUB_CN/TUB_CN_GERAL.xlsx")
    excel_workbook = load_workbook("/content/TUB_CN/TUB_CN_GERAL.xlsx")
    writer = pd.ExcelWriter("/content/TUB_CN/TUB_CN_GERAL.xlsx", engine='openpyxl')

    for ano in lista_anos:
        df = pd.read_csv(f'/content/TUB_CN/{ano}/TUB_CN_{ano}.csv', low_memory=False)

        df = df[df['SG_UF'].isin(lista_ufs)]
        #6. Pós-óbito #1. Caso Novo #4. Não Sabe
        df = df[df['TRATAMENTO'].isin([1,4,6])]
        df = df.groupby(["TRATAMENTO", "SG_UF"]).size().reset_index(name="TOTAL")
        df = df.groupby("SG_UF")["TOTAL"].sum().reset_index(name="TOTAL_CASOS_NOVOS")
        df["ANO"]= ano

        lista_ufs_inv = []
        for index, linha in df.iterrows():
            dict_uf ={
                11:"RO",
                12:"AC",
                13:"AM",
                14:"RR",
                15:"PA",
                16:"AP",
                17:"TO",
                21:"MA",
                22:"PI",
                23:"CE",
                24:"RN",
                25:"PB",
                26:"PE",
                27:"AL",
                28:"SE",
                29:"BA",
                31:"MG",
                32:"ES",
                33:"RJ",
                35:"SP",
                41:"PR",
                42:"SC",
                43:"RS",
                50:"MS",
                51:"MT",
                52:"GO",
                53:"DF",

            }
            lista_ufs_inv.append(dict_uf[linha['SG_UF']])




        df['SIGLA_UF'] = lista_ufs_inv
        df.drop(['SG_UF'], axis=1, inplace=True)
        df = df[["SIGLA_UF", "TOTAL_CASOS_NOVOS", "ANO"]]
        df = df.sort_values(by=['SIGLA_UF'])



        df_ibge = obtem_df_ibge("/content/dados_ibge_projecao_tab_net_2022.xlsx")
        df_ibge = df_ibge[df_ibge['CODIGO_UF'].isin(lista_ufs_inv)]
        df["POP_IBGE"] = df_ibge["POP_ANT"].to_list()


        df["COF_INC_CEM_MIL"] = (df["TOTAL_CASOS_NOVOS"]/df["POP_IBGE"])*100000
        df = df.round({'COF_INC_CEM_MIL': 2})
        df = df[["SIGLA_UF", "TOTAL_CASOS_NOVOS", "POP_IBGE", "COF_INC_CEM_MIL","ANO"]]

        df.to_excel(writer, sheet_name=f'{ano}', index=False)

    writer.close()

In [9]:
cria_df()